# 11 Regularized Model-ElasticNet 실습

## [목적]
### 1. ElasticeNet
* Regularized Linear Model을 활용하여 Overfitting을 방지함
* Hyperparameter lambda를 튜닝할 때 for loop와 GridsearchCV를 활용
### 2. Regularized Linear Models의 경우 X's scaling을 필수적으로 진행해야함
## [Process]
### 1. Define X's &y
### 2. Split Train & Valid dataset
### 3. Modeling
### 4. Model 해석

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.datasets import load_diabetes

In [2]:
import scipy
from sklearn import metrics

def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return sse / X.shape[0]


def adj_r2_score(clf, X, y):
    """Calculate the adjusted :math:`R^2` of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The adjusted :math:`R^2` of the model.
    """
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = metrics.r2_score(y, clf.predict(X))
    return 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))


def coef_se(clf, X, y):
    """Calculate standard error for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of standard errors for the beta coefficients.
    """
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)


def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)


def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print('R-squared:  {0:.6f},    Adjusted R-squared:  {1:.6f},    MSE: {2:.1f}'.format(
        metrics.r2_score(y, clf.predict(X)), adj_r2_score(clf, X, y), sse(clf, X, y)))

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

data_df = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AGE     442 non-null    int64  
 1   SEX     442 non-null    int64  
 2   BMI     442 non-null    float64
 3   BP      442 non-null    float64
 4   S1      442 non-null    int64  
 5   S2      442 non-null    float64
 6   S3      442 non-null    float64
 7   S4      442 non-null    float64
 8   S5      442 non-null    float64
 9   S6      442 non-null    int64  
 10  Y       442 non-null    int64  
dtypes: float64(6), int64(5)
memory usage: 38.1 KB


In [4]:
y = data_df['Y']
X = data_df.drop(columns=['Y'])
X = pd.get_dummies(X, columns=['SEX'])
X.head()

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,59,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,0,1
1,48,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,1,0
2,72,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,0,1
3,24,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,1,0
4,50,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,1,0


In [5]:
idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(len(train_idx), len(valid_idx))

309 133


In [6]:
scaler = MinMaxScaler().fit(X.iloc[train_idx]) # train data로만 훈련
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns=X.columns)
X_scal.head()

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,0.666667,0.573840,0.565217,0.294118,0.297578,0.197368,0.318471,0.562217,0.439394,0.0,1.0
1,0.483333,0.130802,0.362319,0.421569,0.355248,0.618421,0.159236,0.222437,0.166667,1.0,0.0
2,0.883333,0.506329,0.449275,0.289216,0.299885,0.236842,0.318471,0.496578,0.409091,0.0,1.0
3,0.083333,0.286920,0.318841,0.495098,0.517878,0.223684,0.477707,0.572923,0.469697,1.0,0.0
4,0.516667,0.189873,0.565217,0.465686,0.483276,0.381579,0.318471,0.362385,0.333333,1.0,0.0


## [ElasticNet Regression]
* Hyperparameter Tuning using for Loop
* Hyperparameter Tuning using GridSearchCV

## [ElasticeNet Regression Parameters]
  - Package : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
  - alpha : L2-norm Penalty Term
    - alpha : 0 일 때, Just Linear Regression
  - l1_ratio : L1-norm Penalty Term
    - 0 <= l1_ratio <= 1
    - l1_ratio : 1 일 때, Just Ridge Regression
  - fit_intercept : Centering to zero
    - 베타0를 0로 보내는 것 (베타0는 상수이기 때문에)
  - max_iter : Maximum number of interation
    - Loss Function의 LASSO Penalty Term은 절대 값이기 때문에 Gradient Descent와 같은 최적화가 필요함
  - Penalty Term
    - 1 / (2 * n_samples) * ||y - Xw||^2_2 + <u>alpha * l1_ratio * ||w||_1</u> + <u>0.5 * alpha * (1 - l1_ratio) * ||w||^2_2</u>

In [7]:
penalty = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.02, 0.05, 0.1, 0.3, 0.5, 0.7, 1, 5, 10]
l1_ratio = [0.9, 0.7, 0.5, 0.3, 0.1]

In [8]:
for p in penalty:
    for r in l1_ratio:
        model = ElasticNet(alpha=p).fit(X_scal.iloc[train_idx], y.iloc[train_idx])
        score = model.score(X_scal.iloc[valid_idx], y.iloc[valid_idx])
        pred_y = model.predict(X_scal.iloc[valid_idx])
        mse = mean_squared_error(y.iloc[valid_idx], pred_y)
        print(f"Alpha: {p:.7f}, R2: {score:.7f}, MSE: {mse:.7f}, RMSE: {np.sqrt(mse):.7f}")

Alpha: 0.0000100, R2: 0.5302256, MSE: 3084.2148395, RMSE: 55.5357078
Alpha: 0.0000100, R2: 0.5302256, MSE: 3084.2148395, RMSE: 55.5357078
Alpha: 0.0000100, R2: 0.5302256, MSE: 3084.2148395, RMSE: 55.5357078
Alpha: 0.0000100, R2: 0.5302256, MSE: 3084.2148395, RMSE: 55.5357078
Alpha: 0.0000100, R2: 0.5302256, MSE: 3084.2148395, RMSE: 55.5357078
Alpha: 0.0000500, R2: 0.5304135, MSE: 3082.9815630, RMSE: 55.5246032
Alpha: 0.0000500, R2: 0.5304135, MSE: 3082.9815630, RMSE: 55.5246032
Alpha: 0.0000500, R2: 0.5304135, MSE: 3082.9815630, RMSE: 55.5246032
Alpha: 0.0000500, R2: 0.5304135, MSE: 3082.9815630, RMSE: 55.5246032
Alpha: 0.0000500, R2: 0.5304135, MSE: 3082.9815630, RMSE: 55.5246032
Alpha: 0.0001000, R2: 0.5305586, MSE: 3082.0291476, RMSE: 55.5160260
Alpha: 0.0001000, R2: 0.5305586, MSE: 3082.0291476, RMSE: 55.5160260
Alpha: 0.0001000, R2: 0.5305586, MSE: 3082.0291476, RMSE: 55.5160260
Alpha: 0.0001000, R2: 0.5305586, MSE: 3082.0291476, RMSE: 55.5160260
Alpha: 0.0001000, R2: 0.5305586, M

C:\Users\dukim\miniconda3\envs\bootcamp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.878e+05, tolerance: 1.732e+02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dukim\miniconda3\envs\bootcamp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.878e+05, tolerance: 1.732e+02
  model = cd_fast.enet_coordinate_descent(
C:\Users\dukim\miniconda3\envs\bootcamp\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

Alpha: 0.0010000, R2: 0.5301610, MSE: 3084.6392459, RMSE: 55.5395287
Alpha: 0.0010000, R2: 0.5301610, MSE: 3084.6392459, RMSE: 55.5395287
Alpha: 0.0010000, R2: 0.5301610, MSE: 3084.6392459, RMSE: 55.5395287
Alpha: 0.0010000, R2: 0.5301610, MSE: 3084.6392459, RMSE: 55.5395287
Alpha: 0.0100000, R2: 0.5218304, MSE: 3139.3320523, RMSE: 56.0297426
Alpha: 0.0100000, R2: 0.5218304, MSE: 3139.3320523, RMSE: 56.0297426
Alpha: 0.0100000, R2: 0.5218304, MSE: 3139.3320523, RMSE: 56.0297426
Alpha: 0.0100000, R2: 0.5218304, MSE: 3139.3320523, RMSE: 56.0297426
Alpha: 0.0100000, R2: 0.5218304, MSE: 3139.3320523, RMSE: 56.0297426
Alpha: 0.0200000, R2: 0.5092512, MSE: 3221.9186668, RMSE: 56.7619473
Alpha: 0.0200000, R2: 0.5092512, MSE: 3221.9186668, RMSE: 56.7619473
Alpha: 0.0200000, R2: 0.5092512, MSE: 3221.9186668, RMSE: 56.7619473
Alpha: 0.0200000, R2: 0.5092512, MSE: 3221.9186668, RMSE: 56.7619473
Alpha: 0.0200000, R2: 0.5092512, MSE: 3221.9186668, RMSE: 56.7619473
Alpha: 0.0500000, R2: 0.4684468, M

In [9]:
model_best = ElasticNet(alpha=0.02).fit(X_scal.iloc[train_idx], y.iloc[train_idx])
summary(model_best, X_scal.iloc[valid_idx], y.iloc[valid_idx], xlabels=X.columns)

Coefficients:
              Estimate    Std. Error  t value   p value
_intercept   48.093201  3.887779e+08   0.0000  1.000000
AGE          -5.356582  2.437852e+01  -0.2197  0.826424
BMI         105.941275  3.250571e+01   3.2592  0.001421
BP           61.583874  2.889356e+01   2.1314  0.034909
S1           -6.957002  1.673428e+02  -0.0416  0.966902
S2          -11.267900  1.186959e+02  -0.0949  0.924514
S3          -49.678169  7.217783e+01  -0.6883  0.492488
S4           28.961671  5.812461e+01   0.4983  0.619124
S5           94.632519  5.116358e+01   1.8496  0.066607
S6           27.025748  3.479475e+01   0.7767  0.438714
SEX_1         9.756084  3.887779e+08   0.0000  1.000000
SEX_2        -9.517310  3.887779e+08  -0.0000  1.000000
---
R-squared:  0.509251,    Adjusted R-squared:  0.464638,    MSE: 3221.9


In [10]:
# using ElasticNetCV
grid = dict()
grid['alpha'] = penalty
grid['l1_ratio'] = l1_ratio

In [11]:
# define model
model = ElasticNet()
# define search
search = GridSearchCV(model, grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
# perform the search
results = search.fit(X_scal.iloc[valid_idx], y.iloc[valid_idx])
# summarize
print('RMSE: {:.4f}'.format(-results.best_score_))
print('Config: {}'.format(results.best_params_))

RMSE: 56.8906
Config: {'alpha': 0.01, 'l1_ratio': 0.3}


In [12]:
model_best = ElasticNet(alpha=results.best_params_['alpha'],
                        l1_ratio=results.best_params_['l1_ratio']).fit(X_scal.iloc[train_idx], y.iloc[train_idx])
score = model_best.score(X_scal.iloc[valid_idx], y.iloc[valid_idx])
pred_y = model_best.predict(X_scal.iloc[valid_idx])
mse = mean_squared_error(y.iloc[valid_idx], pred_y)
print("Alpha:{0:.7f}, l1_ratio: {1:.7f}, R2:{2:.7f}, MSE:{3:.7f}, RMSE:{4:.7f}".format(results.best_params_['alpha'],
                                                                                   results.best_params_['l1_ratio'],
                                                                                   score, mse, np.sqrt(mse)))
summary(model_best, X_scal.iloc[valid_idx], y.iloc[valid_idx], xlabels=X.columns)

Alpha:0.0100000, l1_ratio: 0.3000000, R2:0.5171797, MSE:3169.8655476, RMSE:56.3015590
Coefficients:
              Estimate    Std. Error  t value   p value
_intercept   45.921027  3.856246e+08   0.0000  1.000000
AGE          -7.326230  2.367229e+01  -0.3095  0.757440
BMI         112.053983  3.220196e+01   3.4797  0.000681
BP           63.835355  2.851909e+01   2.2383  0.026874
S1          -10.286770  1.658942e+02  -0.0620  0.950650
S2          -11.664902  1.176090e+02  -0.0992  0.921143
S3          -51.885195  7.142718e+01  -0.7264  0.468875
S4           26.375923  5.759678e+01   0.4579  0.647748
S5          102.402575  5.065198e+01   2.0217  0.045230
S6           25.791145  3.430041e+01   0.7519  0.453438
SEX_1        10.231459  3.856246e+08   0.0000  1.000000
SEX_2        -9.855686  3.856246e+08  -0.0000  1.000000
---
R-squared:  0.517180,    Adjusted R-squared:  0.473287,    MSE: 3169.9


In [13]:
# BMI, S3, S5 만 LinearRegression 적용
target_column = ['BMI', 'BP', 'S3', 'S5', 'S6']

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
results = LinearRegression().fit(X.iloc[train_idx][target_column], y.iloc[train_idx])

In [16]:
summary(results, X.iloc[valid_idx][target_column], y.loc[valid_idx], xlabels=target_column)

Coefficients:
              Estimate  Std. Error  t value   p value
_intercept -244.912511   71.117961  -3.4438  0.000769
BMI           5.685139    1.257908   4.5195  0.000014
BP            0.724639    0.302647   2.3943  0.018056
S3           -0.668388    0.315558  -2.1181  0.036040
S5           44.553518   10.867197   4.0998  0.000072
S6            0.072288    0.328272   0.2202  0.826051
---
R-squared:  0.504894,    Adjusted R-squared:  0.485402,    MSE: 3250.5
